In [1]:
import json
import os
from pprint import pprint

import tiktoken
from config import EMBEDDING_MODEL
from dotenv import find_dotenv, load_dotenv
from qdrant_client.http.models import PointStruct
from utils import (
    create_collection,
    get_collection_info,
    get_count,
    upsert,
    search,
    embed_text,
)

load_dotenv(find_dotenv())

True

In [14]:
raw_data_path = "../scraper/srb_labor_law_data.json"

In [15]:
with open(raw_data_path, "r", encoding="utf-8") as file:
    raw_data = json.loads(file.read())

## Embedd data

Create JSONL for parallel embedding

In [42]:
filename = "./requests_to_parallel_process.jsonl"
jobs = [
    {
        "model": EMBEDDING_MODEL,
        "input": ". ".join([sample["title"], " ".join(sample["texts"])]),
    }
    for sample in raw_data
]
with open(filename, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [44]:
! python api_request_parallel_processor.py \
  --requests_filepath requests_to_parallel_process.jsonl \
  --save_filepath requests_to_parallel_process_results.jsonl \
  --request_url https://api.openai.com/v1/embeddings \
  --max_requests_per_minute 2500 \
  --max_tokens_per_minute 900000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

Create PointStructures for Qdrant database

In [11]:
embeddings_path = "./requests_to_parallel_process_results.jsonl"
with open(embeddings_path, "r", encoding="utf-8") as file:
    embeddings = []
    for line in file:
        embeddings.append(json.loads(line))

In [12]:
embeddings_lookup = {}
for item in embeddings:
    text = item[0]["input"]
    article_name = text.split(". ")[0]
    embedding = item[1]["data"][0]["embedding"]
    embeddings_lookup[article_name] = {"embedding": embedding, "text": text}

In [16]:
points = []

for id, dictionary in enumerate(raw_data):
    title = dictionary["title"]
    link = dictionary["link"]
    if title in embeddings_lookup:
        embedding, text = (
            embeddings_lookup[title]["embedding"],
            embeddings_lookup[title]["text"],
        )
        points.append(
            PointStruct(
                id=id,
                vector=embedding,
                payload={"title": title, "text": text, "link": link},
            )
        )
    else:
        print(
            f"Warning: No embedding found for title '{title}'. This item will be skipped."
        )

# Create Vector database

In [17]:
collection_name = "labor_law"
create_collection(name=collection_name)

2024-03-23 22:15:21.809 | INFO     | utils:create_collection:30 - Creating collection: labor_law with vector size: 1536.


True

In [18]:
upsert(collection=collection_name, points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [19]:
get_collection_info(collection=collection_name)

{'status': <CollectionStatus.GREEN: 'green'>,
 'optimizer_status': <OptimizersStatusOneOf.OK: 'ok'>,
 'vectors_count': 313,
 'indexed_vectors_count': 0,
 'points_count': 313,
 'segments_count': 2,
 'config': {'params': {'vectors': {'size': 1536,
    'distance': <Distance.COSINE: 'Cosine'>,
    'hnsw_config': None,
    'quantization_config': None,
    'on_disk': None},
   'shard_number': 1,
   'sharding_method': None,
   'replication_factor': 1,
   'write_consistency_factor': 1,
   'read_fan_out_factor': None,
   'on_disk_payload': True,
   'sparse_vectors': None},
  'hnsw_config': {'m': 16,
   'ef_construct': 100,
   'full_scan_threshold': 10000,
   'max_indexing_threads': 0,
   'on_disk': False,
   'payload_m': None},
  'optimizer_config': {'deleted_threshold': 0.2,
   'vacuum_min_vector_number': 1000,
   'default_segment_number': 0,
   'max_segment_size': None,
   'memmap_threshold': None,
   'indexing_threshold': 20000,
   'flush_interval_sec': 5,
   'max_optimization_threads': None

In [20]:
get_count(collection=collection_name)

313

# Search the Vector database 

In [21]:
query = "Koliko dana godišnjeg odmora zaposleni u Srbiji ima po zakonu?"

In [22]:
response = embed_text(query)

In [23]:
query_vector = response.data[0].embedding

In [25]:
response = search(
    collection=collection_name, query_vector=query_vector, with_vectors=True
)

In [27]:
[point.payload for point in response]

[{'link': 'https://www.paragraf.rs/propisi/zakon_o_radu.html#clan_69',
  'text': 'Član 69. U svakoj kalendarskoj godini zaposleni ima pravo na godišnji odmor u trajanju utvrđenom opštim aktom i ugovorom o radu, a najmanje 20 radnih dana. Dužina godišnjeg odmora utvrđuje se tako što se zakonski minimum od 20 radnih dana uvećava po osnovu doprinosa na radu, uslova rada, radnog iskustva, stručne spreme zaposlenog i drugih kriterijuma utvrđenih opštim aktom ili ugovorom o radu.',
  'title': 'Član 69'},
 {'link': 'https://www.paragraf.rs/propisi/zakon_o_radu.html#clan_68',
  'text': 'Član 68. Zaposleni ima pravo na godišnji odmor u skladu sa ovim zakonom. Zaposleni stiče pravo na korišćenje godišnjeg odmora u kalendarskoj godini posle mesec dana neprekidnog rada od dana zasnivanja radnog odnosa kod poslodavca. Pod neprekidnim radom smatra se i vreme privremene sprečenosti za rad u smislu propisa o zdravstvenom osiguranju i odsustva sa rada uz naknadu zarade. Zaposleni ne može da se odrekne 